In [1]:
from arnie.utils import evaluate_L1_shape_score
evaluate_L1_shape_score(".(.).",[0.2,0.5,0.32,0.43,0.4])

def evaluate_L1_shape_score(s,shape):
    score = 0
    for c,react in zip(s,shape):
        if (c=="." and react>0.25) or (c!="." and react<0.5):
            score += 1
    return score/len(s)

ImportError: cannot import name 'evaluate_L1_shape_score' from 'arnie.utils' (/home/gnye8/Desktop/PK_research/arnie/utils.py)